In [42]:
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

import warnings
warnings.filterwarnings('ignore')

In [43]:
df = pd.read_csv('./src/네이버종토방댓글_카카오_15000_태그토큰화_라벨.csv')
df = df.drop_duplicates('토큰화댓글')
df = df.reset_index(drop=True)
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글,토큰화댓글,공포탐욕
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법,"['개미', '조련', '하다', '법']",0.5
1,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']",0.5
2,2022-05-03,안티 살발하다,34,2,0,안티 살발하다,"['안티', '살발']",0.0
3,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']",0.5
4,2022-05-03,곡소리 났구나.,66,0,0,곡소리 났구나,"['곡', '소리', '나다']",0.5
...,...,...,...,...,...,...,...,...
207270,2017-10-18,여기서 털리는 빠가사리들은 나가죽어라,693,6,6,여기서 털리는 빠가사리들은 나가죽어라,"['여기', '털리다', '빠가사리', '나가다', '죽다']",0.0
207271,2017-10-18,카카오 네이버 제휴!,935,2,3,카카오 네이버 제휴,"['카카오', '네이버', '제휴']",0.5
207272,2017-10-18,어차피 또 폭등할텐데 떨필요있나 기회주면...,795,15,3,어차피 또 폭등할텐데 떨필요있나 기회주면,"['어차피', '또', '폭등', '하다', '떨다', '필요', '있다', '회주']",1.0
207273,2017-10-18,오늘 물렸다고 생각하는 사람들은 도대체 ...,961,3,2,오늘 물렸다고 생각하는 사람들은 도대체,"['오늘', '물리다', '생각', '하다', '사람', '도대체']",0.5


In [45]:
tokenizer = Tokenizer(num_words=30000, oov_token = True)
tokenizer.fit_on_texts(df['토큰화댓글'])
df['토큰'] = tokenizer.texts_to_sequences(df['토큰화댓글'])
df
# saving
with open('./src/mytokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
df_test = df[df['공포탐욕'] == 0.5]
df_test = df_test.reset_index(drop=True)
df_test

,날짜,댓글,조회수,좋아요,싫어요,한글댓글,토큰화댓글,공포탐욕,토큰
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법,"['개미', '조련', '하다', '법']",0.5,"[7, 12453, 3, 404]"
1,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']",0.5,"[2, 1591, 27, 655]"
2,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']",0.5,"[135, 3781, 7, 16127, 273]"
3,2022-05-03,곡소리 났구나.,66,0,0,곡소리 났구나,"['곡', '소리', '나다']",0.5,"[778, 304, 113]"
4,2022-05-03,증시버블♡개구리탕 아시나?,90,4,1,증시버블 개구리탕 아시나,"['증시', '버블', '개구리', '탕', '아시']",0.5,"[222, 726, 4734, 1996, 1036]"
...,...,...,...,...,...,...,...,...,...
175565,2017-10-18,■낼 셀트로 가삐까?,480,3,0,낼 셀트로 가삐까,"['낼', '셀트', '가다', '삐', '끄다']",0.5,"[124, 595, 9, 3387, 159]"
175566,2017-10-18,■나도 내일 나가삐까?,580,4,1,나도 내일 나가삐까,"['나다', '내일', '나가다', '삐', '끄다']",0.5,"[113, 24, 207, 3387, 159]"
175567,2017-10-18,카카오 네이버 제휴!,935,2,3,카카오 네이버 제휴,"['카카오', '네이버', '제휴']",0.5,"[2, 22, 4911]"
175568,2017-10-18,오늘 물렸다고 생각하는 사람들은 도대체 ...,961,3,2,오늘 물렸다고 생각하는 사람들은 도대체,"['오늘', '물리다', '생각', '하다', '사람', '도대체']",0.5,"[4, 94, 95, 3, 49, 663]"


In [9]:
df_train = df[df['공포탐욕'] != 0.5]
df_train = df_train.reset_index(drop=True)
df_train

,날짜,댓글,조회수,좋아요,싫어요,한글댓글,토큰화댓글,공포탐욕,토큰
0,2022-05-03,안티 살발하다,34,2,0,안티 살발하다,"['안티', '살발']",0.0,"[46, 9126]"
1,2022-05-03,카카오는 왜 이렇게 국민들의 안티기업이 ...,81,3,2,카카오는 왜 이렇게 국민들의 안티기업이,"['카카오', '왜', '국민', '안티', '기업']",0.0,"[2, 30, 143, 46, 63]"
2,2022-05-03,오늘 상승은,407,0,1,오늘 상승은,"['오늘', '상승']",1.0,"[4, 58]"
3,2022-05-03,오늘 왜 상한가 쳤냐?,168,0,1,오늘 왜 상한가 쳤냐,"['오늘', '왜', '상한', '치다']",1.0,"[4, 30, 123, 68]"
4,2022-05-03,시간외 폭등중이네 ㄷㄷㄷ 뭔일있나 ㄷㄷㄷ,1239,14,4,시간외 폭등중이네 뭔일있나,"['시간', '외', '폭등', '일', '있다']",1.0,"[121, 128, 213, 52, 14]"
...,...,...,...,...,...,...,...,...,...
31700,2017-10-18,낼 무조건 급등이란다 10퍼만 오르자!!...,929,8,2,낼 무조건 급등이란다 퍼만 오르자,"['낼', '무조건', '급등', '퍼', '오르다']",1.0,"[124, 235, 278, 566, 10]"
31701,2017-10-18,■낼 셀트 폭등한다,975,1,4,낼 셀트 폭등한다,"['낼', '셀트', '폭등', '하다']",1.0,"[124, 595, 213, 3]"
31702,2017-10-18,■셀트는 낼도 폭등이다,623,0,4,셀트는 낼도 폭등이다,"['셀', '틀다', '낼', '폭등']",1.0,"[1473, 1099, 124, 213]"
31703,2017-10-18,여기서 털리는 빠가사리들은 나가죽어라,693,6,6,여기서 털리는 빠가사리들은 나가죽어라,"['여기', '털리다', '빠가사리', '나가다', '죽다']",0.0,"[79, 310, 12007, 207, 312]"


In [18]:
train = pad_sequences(df_train['토큰'], maxlen=15)

In [19]:
print(train.shape)
train

(31705, 15)


array([[    0,     0,     0, ...,     0,    46,  9126],
       [    0,     0,     0, ...,   143,    46,    63],
       [    0,     0,     0, ...,     0,     4,    58],
       ...,
       [    0,     0,     0, ...,  1099,   124,   213],
       [    0,     0,     0, ..., 12007,   207,   312],
       [    0,     0,     0, ...,  1360,    14,  4709]])

In [23]:
label = df_train['공포탐욕']
label

0        0.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
31700    1.0
31701    1.0
31702    1.0
31703    0.0
31704    1.0
Name: 공포탐욕, Length: 31705, dtype: float64

In [24]:
model = Sequential()
model.add(Embedding(30000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
hist = model.fit(train, label, batch_size=32, epochs=5)

Epoch 1/5
991/991 [==============================] - 94s 93ms/step - loss: 0.0525 - accuracy: 0.9808
Epoch 2/5
991/991 [==============================] - 89s 89ms/step - loss: 0.0057 - accuracy: 0.9979
Epoch 3/5
991/991 [==============================] - 91s 92ms/step - loss: 0.0036 - accuracy: 0.9988
Epoch 4/5
991/991 [==============================] - 91s 91ms/step - loss: 0.0030 - accuracy: 0.9990
Epoch 5/5
991/991 [==============================] - 88s 89ms/step - loss: 0.0015 - accuracy: 0.9995


In [27]:
test = pad_sequences(df_test['토큰'], maxlen=15)

In [28]:
test

array([[    0,     0,     0, ..., 12453,     3,   404],
       [    0,     0,     0, ...,  1591,    27,   655],
       [    0,     0,     0, ...,     7, 16127,   273],
       ...,
       [    0,     0,     0, ...,     2,    22,  4911],
       [    0,     0,     0, ...,     3,    49,   663],
       [    0,     0,     0, ...,     0,   474,   239]])

In [29]:
pred = model.predict(test)

In [30]:
pred

array([[0.9919636 ],
       [0.78963584],
       [0.9915315 ],
       ...,
       [0.57868624],
       [0.09763294],
       [0.05635315]], dtype=float32)

In [31]:
df_test['예측'] = pred

In [32]:
df_test[['댓글','예측']]

,댓글,예측
0,개미들 조련하는법,0.991964
1,카카오는 끝물입니다 사실상,0.789636
2,이런 호로개미지옥탕을 만드신,0.991531
3,곡소리 났구나.,0.999996
4,증시버블♡개구리탕 아시나?,0.699422
...,...,...
175565,■낼 셀트로 가삐까?,0.149885
175566,■나도 내일 나가삐까?,0.192151
175567,카카오 네이버 제휴!,0.578686
175568,오늘 물렸다고 생각하는 사람들은 도대체 ...,0.097633


In [33]:
for i in range(10000):
    print(df_test['토큰화댓글'][i],'==',df_test['예측'][i])

['개미', '조련', '하다', '법'] == 0.9919636
['카카오', '끝물', '이다', '사실'] == 0.78963584
['이렇다', '호로', '개미', '지옥탕', '만들다'] == 0.9915315
['곡', '소리', '나다'] == 0.99999595
['증시', '버블', '개구리', '탕', '아시'] == 0.6994221
['중국', '기업', '그렇다'] == 0.17657357
['팔', '처', '묵다', '개미', '새', '말', '차다', '많다'] == 0.9998989
['필독', '리먼', '사태', '재현', '패턴', '등장'] == 0.9988384
['휴지', '주식', '가가', '오'] == 0.039648175
['내일', '카카오', '프로', '내리다', '날'] == 0.99630713
['카카오', '중', '장기', '점', '가격', '얼마'] == 0.13279378
['버핏', '애플', '더', '담다'] == 0.99780035
['얘', '나스닥', '상장', '하다'] == 0.10961944
['불안하다', '신분'] == 0.8587393
['주식', '못', '하다', '사람', '특징'] == 0.98075926
['장', '투하', '시', '분'] == 0.9099721
['만약', '강하다', '깨다', '대면'] == 0.09246233
['앵카', '자석', '들다', '왜', '자꾸', '카카오', '후', '비파'] == 0.9864323
['카카오', '유투브', '하부', '만원', '년뒤', '온'] == 0.9953768
['악', '악', '규정', '되어다', '처벌', '되다', '법'] == 0.99999946
['머스크', '한국', '방문', '하다'] == 0.99917305
['내일', '자', '사주', '일인'] == 0.012759864
['주식시장', '놀이터'] == 0.9562427
['나쁘다', '않다'] == 0.97794

['산봉우리', '차트', '이쁘다'] == 0.5777611
['일단', '전부', '손절침'] == 0.9213516
['네이버', '추정', '가다'] == 0.68724155
['힘', '있다', '때', '달리다'] == 0.02347061
['지주', '차트', '비슷하다', '만원', '가다', '만원'] == 0.99362004
['수출', '외화', '벌이', '하다', '있다', '이', '회사'] == 0.9998363
['아니다', '이', '기업'] == 0.50589937
['이', '가격'] == 0.8182672
['카카오', '너', '믿다'] == 0.8277298
['크다', '아니다', '척', '발버둥', '치다', '보다'] == 0.2124359
['개', '크다', '어디가'] == 0.2822023
['이르다', '문어', '머리', '평생', '월급', '못', '받다', '아니다'] == 0.99999183
['이렇다', '국', '내용', '소상', '공기업', '총', '조'] == 0.9704431
['배당', '라비', '양심', '기업'] == 0.9302342
['생건', '밑', '개', '잡다', '타령', '하다'] == 0.5307494
['개', '크다', '일인', '하다'] == 0.049450636
['층', '어떻다'] == 0.48448205
['훼', '이크', '겁나다', '들어오다'] == 0.5746718
['대폭', '크게', '방', '빠지다'] == 0.98231363
['유대', '자본', '수중'] == 0.047037512
['흐미', '물렷'] == 0.03710708
['카카오', '모빌리티', '콜', '몰다', '발목', '잡'] == 0.99965656
['아침', '자다', '때'] == 0.22354722
['우리나라', '개', '잡다', '랭킹', '순위표'] == 0.6180924
['레알', '가스', '팅', '당하다', '기분'] == 0.59

['층', '오다', '하다'] == 0.92250717
['들다', '평단', '얼마'] == 0.52376986
['시대', '들다', '처절하다', '미래', '맞이', '하다'] == 0.99939865
['만원', '마지막', '찬스', '곧', '끝나다', '감'] == 0.9998877
['카카오', '대체', '기업', '누가', '안만드'] == 0.38869148
['카카오', '봉', '김선', '달'] == 0.2217134
['힘', '못', '쓰다'] == 0.33834648
['젤', '빡치다'] == 0.99430025
['수급', '문제'] == 0.6697253
['개한민국', '플랫폼', '반토막'] == 0.10566163
['나스닥', '밀리', '우수수'] == 0.031330675
['조심', '해'] == 0.558036
['물', '타다', '하다'] == 0.57668316
['대다', '깨다', '끝나다', '정권', '교체', '되다', '월'] == 0.9999647
['배당', '은행', '금리', '낮다'] == 0.95603085
['엔진', '만세'] == 0.9331019
['만원', '언제', '오냐'] == 0.55694383
['만원', '사장'] == 0.6706496
['년', '동안', '주주', '능', '당하다', '있다'] == 0.2991405
['한국', '증시', '진짜', '답', '없다'] == 0.96516335
['라이', '쌩', '아치', '새끼', '김범수'] == 0.0006394088
['카카오', '대치', '목표'] == 0.93744576
['요즘', '국장', '정해지다'] == 0.97814995
['세상', '슬프다', '하다'] == 0.26940745
['카카오', '절대', '사선', '안', '돼다', '이유', '가지'] == 0.14571983
['층', '계', '신분'] == 0.88454986
['미국', '해운', '주', '한국', 

['증권사', '들다', '보고서', '내다', '공매도'] == 0.98266923
['흡혈귀', '놈'] == 0.25270382
['쌍용', '인수', '하다', '않다', '답', '없다'] == 0.9262816
['주식', '정보', '공유', '카페'] == 0.9951658
['고장', '주가'] == 0.38568497
['카카오', '유튜브', '참고'] == 0.99949837
['개', '떡', '같다'] == 0.42685804
['차', '원'] == 0.74701977
['부터', '모으다'] == 0.91581273
['오다', '물', '타다'] == 0.76901746
['문어', '믿다', '머리털', '사람', '믿다'] == 0.9990634
['손', '절하', '길', '다행', '이다'] == 0.9997305
['쓰레기', '개', '잡다', '치다', '물리다'] == 0.120986015
['만', '만들다', '이', '대머리'] == 0.7714821
['크다', '톡'] == 0.031902462
['힘드다', '원', '입성'] == 0.58886147
['증시', '버블', '현금화', '필수'] == 0.07741302
['카카오', '왜', '빠지다', '이유', '있다'] == 0.99525857
['저', '층', '사람', '있다'] == 0.7805496
['다시', '지옥', '가다'] == 0.8912916
['카카오', '가다', '주가', '만원', '되다', '때'] == 0.50036824
['가온', '미디어', '자회사'] == 0.99177444
['알짜', '빼', '간다', '언론플레이', '하다', '주가', '악'] == 0.9999943
['공매도', '기승'] == 0.638883
['카카오', '답', '없다'] == 0.25024152
['대한민국', '사라지다', '되다', '인간'] == 0.9998392
['이', '대기업'] == 0.21909931
['어

['하루', '종일', '싸다'] == 0.7635893
['반토막', '카카오', '너', '이렇다'] == 0.9751725
['서울', '강남', '판자촌', '해도', '클라스'] == 0.9999387
['지지', '부진하다', '때'] == 0.4752577
['아니다', '도주', '알다', '늘다', '왜', '카카오'] == 0.9508232
['카카오', '만', '가즈'] == 0.9307857
['껌', '지네'] == 0.95244503
['대표', '란', '새끼', '돌아가다', '고치'] == 0.98673356
['개', '잡다', '탈출', '하다'] == 0.7493326
['투자', '하다', '미국', '주식'] == 0.7712139
['살짝', '굼뜬데'] == 0.57208014
['요즘', '왜', '그렇다'] == 0.989249
['오다', '개지', '럴'] == 0.6751106
['오늘', '만원', '돌', '판다'] == 0.69868267
['지수', '받치다', '때', '좀', '가라', '만', '개미', '주'] == 0.9982617
['음', '내리다', '같다'] == 0.9955088
['오다', '개', '팔다', '거', '공매도'] == 0.98075294
['만', '넘기다', '오늘'] == 0.643155
['속보', '억', '성과급', '달라', '법정'] == 0.9756891
['안착', '하자'] == 0.99344766
['부르다', '찌', '타령', '주가', '집중', '하다'] == 0.017470747
['청와대', '김정숙', '옷', '특활비', '지출', '안', '하다'] == 0.93551785
['오다', '김정숙', '까르띠', '브로치', '빛나다'] == 0.7944852
['오늘', '아침', '못', '담다', '이', '화근'] == 0.9982321
['총', '오라', '문자', '하나', '안', '오다'] == 0.9523021


...

['평단', '다시', '가다'] == 0.83511317
['정권', '바뀌다', '랠리', '시작'] == 0.7495073
['좌파', '들다'] == 0.9004401
['크다', '역시', '십만', '간다'] == 0.7817837
['카카오', '지금', '들어가다', '되다'] == 0.9971532
['해안', '만', '간다', '찬성', '못', '간다'] == 0.9999998
['주식', '수학'] == 0.46140462
['층', '사람', '있다', '살', '줄다'] == 0.51595044
['뭐', '만', '찍엇네'] == 0.22145653
['또', '개인'] == 0.072751075
['속보', '위선', '정권', '물러나다', '때', '월'] == 0.0024639368
['카카오', '오늘', '큰일', '하다'] == 0.16323105
['여의도', '애널', '지인'] == 0.15435305
['송영길', '카카오', '안', '건드리다', '이번'] == 0.5676466
['꼬만올', '정말', '만', '느끼다', '보다'] == 0.99895597
['내일', '스톡옵션', '만주', '상장', '하다'] == 0.02899158
['좌파', '끝나다', '불', '되다'] == 0.99998057
['대한민국', '만세', '카카오', '만세'] == 0.9765153
['건희', '관련', '주'] == 0.9944932
['개미', '왜', '비싸다', '사서', '장기간', '고통', '받다'] == 0.9997642
['오늘', '개인', '파다', '비융'] == 0.40368095
['좌파', '정권', '끝나다'] == 0.9926326
['건지다', '법사', '혜안'] == 0.7809228
['통', '카카오'] == 0.94588566
['여기', '층'] == 0.7374032
['그렇다', '왜'] == 0.9802624
['낼', '띠기', '콜', '가즈'] == 0.

['공매도', '누르다', '검다', '머리', '외늠들', '치다'] == 0.0019716024
['아쉽다', '추다', '못', '하다'] == 0.022491544
['긴급', '속보', '러시아군', '철수'] == 0.9446099
['참고', '하다', '바라다'] == 0.93856454
['오늘', '같다', '장날'] == 0.9754621
['시진핑', '경제', '지원', '지시'] == 0.99975055
['제이', '피', '모간', '그만하다'] == 0.99798113
['엔진', '사라', '하다', '제', '내'] == 0.9995699
['종필', '천주', '투하'] == 0.9001184
['말', '올리다', '잉'] == 0.101849526
['나무', '카카', '오지'] == 0.30564255
['감자', '하다'] == 0.5733491
['한국', '개판', '분전', '도박', '판', '이유'] == 0.7230799
['오늘', '강', '보합', '마감', '되다', '듯'] == 0.9999895
['러시아', '침략', '크다', '탓', '일본', '침'] == 0.6904677
['중', '보유', '유효'] == 0.74320865
['특징', '주', '카카오', '어쩐지', '늦다', '잡히다'] == 0.9859159
['만원', '강하다', '뚫다'] == 0.37748635
['차', '바이오', '텍', '은', '흑자', '나오다', '인기', '없다', '넹'] == 0.9969095
['좀', '더', '노력', '하다', '이', '따라잡다'] == 0.8560111
['대선', '끝나다', '간다', '카카오', '선후'] == 0.9999976
['유영', '주의보'] == 0.106559604
['금요일', '날', '삿', '덥다'] == 0.19640508
['이제', '평단', '오다'] == 0.5738672
['개관', '잡', '넘다', '들다', '무지'

['고', '주', '대한', '우화'] == 0.96302104
['기적', '만', '가지', '않다', '여'] == 0.96231997
['대선', '후보', '콕', '찌다'] == 0.038945705
['공매도', '적당하다', '치다'] == 0.2972169
['일', '남다'] == 0.030965
['코스피', '인기', '검색어', '위'] == 0.9754331
['껍데기', '회사', '분노하다', '주주', '분'] == 0.64005184
['정권', '일', '남다'] == 0.007819533
['위드', '재앙'] == 0.83930206
['우크라이나', '포', '쏘다'] == 0.0005838275
['포탄', '쏘다', '내리다'] == 0.08389729
['고오', '정보'] == 0.9917768
['오늘', '일단', '내리다'] == 0.99835634
['카카오', '매출'] == 0.4920129
['바보', '윤석열', '대통령', '삼성', '망하다'] == 0.9980344
['러우', '기', '만해', '같다', '사드', '배치'] == 0.030769229
['경영', '진아', '어서', '다음', '획기', '발표', '하다'] == 0.98539376
['기간', '조정', '년'] == 0.0033063293
['우크라이나', '카카오', '방해', '하다'] == 0.00021791458
['오늘', '자다', '이루다', '밤'] == 0.49401468
['지금', '평가'] == 0.9925239
['차라리', '구글', '사라'] == 0.9925662
['범수', '튀다', '껍데기', '남다'] == 0.0206455
['개', '기금', '외놈', '합작'] == 0.9981996
['카카오', '나락가'] == 0.09322086
['러시아', '돈바스', '구역', '전군', '진군'] == 0.013878763
['어제', '더듬다', '당', '택시', '규제', '

['봉', '선'] == 0.27890545
['배당', '공시'] == 0.9555608
['홀짝', '싸움'] == 0.8070861
['글로벌', '위기'] == 0.59605116
['전쟁', '나다', '월', '일', '금융시장', '초'] == 0.0021440387
['러시아', '우크라이나', '뉴스'] == 0.0010282397
['훈련', '마침', '전쟁', '발포', '백악관', '발'] == 0.005760163
['이번', '만원'] == 0.5592127
['러시아', '크다', '침공', '준비', '끝', '푸틴', '결'] == 0.000529021
['전쟁', '나다', '악', '재해', '소라'] == 0.99833447
['식이', '문제'] == 0.9247519
['전쟁', '나', '일단', '하다'] == 0.96672297
['비트', '빠지다'] == 0.87071425
['러시아', '달', '일', '우크라이나', '침공', '하다'] == 6.2166387e-06
['누가', '요즘', '전쟁', '하나', '무식하다'] == 0.090608954
['나스닥', '개떡락'] == 0.5089621
['차', '대전', '미국', '좌빨', '민주당', '일으키다'] == 0.99922687
['진짜', '전쟁', '터지다'] == 0.0005455017
['서학', '공부', '하다', '간', '형', '태', '전화', '한번', '해보다'] == 0.9946178
['속보', '침공'] == 0.7938911
['종필', '이형', '신나다', '팔', '버리다'] == 0.8702594
['골든', '크로스'] == 0.3857158
['연기금', '제발'] == 0.99085474
['지금', '국제', '정세', '안좋다', '금리', '인상', '추이'] == 0.66399753
['배당금', '올려주다'] == 0.24141356
['좀', '갈다', '하니', '시장', '개망'] ==

['만원', '때', '최저임금'] == 0.16464996
['역린', '건드리다', '격노하다', '대통령', '선정'] == 0.9801613
['카카오', '지금', '점', '맞다'] == 0.9993937
['상여금', '주식', '팔', '주다'] == 0.9869909
['카카오', '남궁훈', '주가', '만원', '되다', '때'] == 0.689086
['껍데기', '가라'] == 0.22690892
['만원', '만들다', '방법', '갈다', '켜다'] == 0.5108605
['년', '벌다', '주가'] == 0.02495417
['시즌', '지나다'] == 0.15691936
['소각', '하다', '모자라다', '판', '뭐', '이번', '놈'] == 0.024680525
['자꾸', '이상하다', '소리'] == 0.86212194
['이미지', '개선'] == 0.40619135
['이', '와중', '껌값', '자사', '주로', '하다'] == 0.99929094
['앞', '최저임금', '뒤', '상여금'] == 0.85398245
['주', '자', '사주', '처분', '공시'] == 0.98827887
['나다', '카카오', '물리다'] == 0.75951755
['주가', '지랄', '상여', '주다', '자기', '주식', '처분'] == 0.6469952
['카카오', '엔터', '상장', '하다', '카카오', '가치', '또'] == 0.014485508
['선', '넘다'] == 0.19318086
['독과점', '하다'] == 0.28512987
['오다', '이렇다', '쓰레기', '회사'] == 0.9998756
['오다', '조'] == 0.6794014
['마넌', '가즈', '넌', '하다', '오'] == 0.97387445
['하루', '개', '월동', '안'] == 0.94329613
['이제', '서다'] == 0.63828486
['자', '사주', '계속', '파다'] == 0.

In [35]:
model.save('./src/mymodel1.h5')

In [40]:
import pickle

# saving
with open('./src/mytokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

In [41]:
tokenizer